# Asystent AI - PDF RAG
Po dwóch udanych misjach – rozwikłaniu tajemnicy przeżycia na Titanicu i odczytaniu ręcznych notatek o pokojach – myślałeś, że zasłużysz na odpoczynek. Jednak telefon od tajemniczego zleceniodawcy rozbrzmiał ponownie, tym razem z jeszcze pilniejszą sprawą.

"Wasze poprzednie modele były kluczowe," – rozpoczął głos w słuchawce – "ale to dopiero wstęp do prawdziwego wyzwania. Nasi agenci przechwycili poufny szpont dokumentów. To nie są zwykłe papiery; to zestaw od 3 do 5 artykułów naukowych w formacie PDF, które zawierają fragmenty przełomowych badań. Informacje są niekompletne, celowo pomieszane i zaszyfrowane."

### Zadanie
Twoim celem będzie stworzenie asystenta RAG - Retrival Augmented Generation na podstawie kilku plików PDF z artykułami naukowymi na wybrany przez siebie temat (3-5 artykułów).
### Poniżej znajdziesz pytania, które mogą być pomocne w zadaniu:
- Czy model poprawnie odpowiada na pytania ?
- Jakie są etapy przetwarzania dokumentu tekstowego ?
- Jaki wynik jest "dobry" ?
- Co robi mój asystent AI ? Jak on działa ? Jak tworzona jest odpowiedź ?
- Czy model poprawnie wybiera fragment dokumentu ? Czy poprawnie korzysta z bazy ?
- Jaką rolę w naszym systemie pełni baza wektorowa ?

### Wymagania
- Przygotuj artykuły naukowe i umieść je w folderze na dysku
- Stwórz wektorową bazę danych (można użyć innego systemu niż FAISS)
- Zapełnij bazę embeddingami z modelu wybranego przez Ciebie z [Sentence Similarity Models](https://huggingface.co/models?pipeline_tag=sentence-similarity&sort=trending) z HuggingFace Hub - innego niż podany w notebooku wzorcowym
- Zainicjuj wybrany model QA z dostępnych [Question Answering Models](https://huggingface.co/models?pipeline_tag=question-answering&sort=trending) z HuggingFace Hub - również innego niż w notebooku wzorcowym
- Stwórz chaina odpowiadania na pytania i dodaj mechanizm pętli - tak, żeby można było przeprowadzać swobodną rozmowę z asystentem

**Dodatkowe wymagania**
- Mechanizm pamięci konwersacji - tak, żeby model pamiętał poprzednią część konwersacji
- Mechanizm braku halucynacji (bez cyberpsychozy) - jeśli model nie znajdzie odpowiedzi w bazie, informuje o tym
- Cytowanie dokumentu, z którego model pobrał informacje

Niezmiennie, zadbaj o czytelność kodu i nazewnictwo zmiennych. Jeśli jakiś wycinek kodu się powtarza, to wyodrębnij go do funkcji. Postaraj się zamieszczać swoje wnioski w postaci komentarza Markdown.






# Rozkład jazdy
Przejdziemy przez etapy przetwarzania tekstu: wczytanie i przygotowanie dokumentów, zakodowanie ich za pomocą modelu typu BERT w postaci embeddingów, zapisanie embeddingów w bazie wektorowej, a następnie wykorzystanie LLM do "odpytywania" naszego tekstu. Po tym etapie rozszerzymy system o mechanizm pętli.

## przygotowane artykuły
Nie wiem czy to dobrze ale umieszczę pliki lokalnie bo nie chce mi się bawić z google drive, sorry jeśli to jest źle. Będą to artykuły na temat optymalizacji oraz sieci grafowych umieszczone w folderze documents.

## instalacja potrzebnych bibliotek

In [1]:
!pip install langchain sentence-transformers faiss-cpu pypdf transformers torch langchain-community

Defaulting to user installation because normal site-packages is not writeable


In [1]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.llms import HuggingFacePipeline
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline, AutoModelForSeq2SeqLM
from langchain.chains import RetrievalQA
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
import os
from langchain.llms import HuggingFacePipeline
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline, AutoModelForSeq2SeqLM
from langchain.chains import RetrievalQA
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory

## wczytanie artykułów

In [2]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=300,
    chunk_overlap=100,
    length_function=len,
    separators=["\n\n", "\n", " ", ""]
)

path = "documents"
docs = []
for file_name in os.listdir(path):
    print(f"Przetwarzany dokument: {file_name}")
    loader = PyPDFLoader(f"{path}/{file_name}")
    pages = loader.load()
    docs += text_splitter.split_documents(pages)

for idx,doc in enumerate(docs):
  print(f"[{idx}].{doc}\n\n")

Ignoring wrong pointing object 8 0 (offset 0)
Ignoring wrong pointing object 26 0 (offset 0)
Ignoring wrong pointing object 32 0 (offset 0)
Ignoring wrong pointing object 45 0 (offset 0)
Ignoring wrong pointing object 55 0 (offset 0)
Ignoring wrong pointing object 98 0 (offset 0)
Ignoring wrong pointing object 111 0 (offset 0)
Ignoring wrong pointing object 121 0 (offset 0)
Ignoring wrong pointing object 129 0 (offset 0)
Ignoring wrong pointing object 157 0 (offset 0)
Ignoring wrong pointing object 161 0 (offset 0)
Ignoring wrong pointing object 187 0 (offset 0)
Ignoring wrong pointing object 195 0 (offset 0)
Ignoring wrong pointing object 220 0 (offset 0)
Ignoring wrong pointing object 248 0 (offset 0)
Ignoring wrong pointing object 255 0 (offset 0)
Ignoring wrong pointing object 266 0 (offset 0)
Ignoring wrong pointing object 274 0 (offset 0)
Ignoring wrong pointing object 277 0 (offset 0)
Ignoring wrong pointing object 291 0 (offset 0)
Ignoring wrong pointing object 299 0 (offset 0)

Przetwarzany dokument: doc_1.pdf
Przetwarzany dokument: doc_2.pdf
Przetwarzany dokument: doc_3.pdf
[0].page_content='1' metadata={'producer': 'macOS Version 10.14.2 (Build 18C54) Quartz PDFContext', 'creator': 'Word', 'creationdate': "D:20191104204801Z00'00'", 'title': 'Microsoft Word - Quantum_Bridge_Analytics_1_Qubo_for_Oct 31.docx', 'moddate': "D:20191104204801Z00'00'", 'keywords': '', 'aapl:keywords': '[]', 'source': 'documents/doc_1.pdf', 'total_pages': 46, 'page': 0, 'page_label': '1'}


[1].page_content='Quantum Bridge Analytics I: A Tutorial on Formulating and Using QUBO Models  Fred Glover1, Gary Kochenberger2, Yu Du2   Abstract   Quantum Bridge Analytics relates generally to methods and systems for hybrid classical-quantum computing, and more particularly is devoted to developing tools for' metadata={'producer': 'macOS Version 10.14.2 (Build 18C54) Quartz PDFContext', 'creator': 'Word', 'creationdate': "D:20191104204801Z00'00'", 'title': 'Microsoft Word - Quantum_Bridge_Analy

## zapisanie baza danych wektorowej

In [3]:
# Model embeddings z Hugging Face
model_name = "sentence-transformers/all-mpnet-base-v2"
embeddings = HuggingFaceEmbeddings(model_name=model_name)
embeddings

C:\Users\jakub\AppData\Local\Temp\ipykernel_18828\1527357035.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name=model_name)


HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 384, 'do_lower_case': False, 'architecture': 'MPNetModel'})
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='sentence-transformers/all-mpnet-base-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [4]:
# Tworzenie i zapis bazy FAISS
db = FAISS.from_documents(docs, embeddings)
db.save_local("faiss_index_hf")

## wyszukiwanie podobnych materiałów

In [5]:
# Wczytanie bazy danych
db = FAISS.load_local("faiss_index_hf", embeddings, allow_dangerous_deserialization=True)

# Wyszukiwanie podobnych fragmentów
query = "Tell me what is a qubo ?"
similar_docs = db.similarity_search(query, k=3)  # top 3 wyniki

for doc in similar_docs:
    print(doc.page_content[:300] + "...\n---")

Comment on the formal classification of QUBO models and their solution: QUBO models belong to a class of problems known to be NP-hard.  The practical meaning of this is that exact solvers designed to find “optimal” solutions (like the commercial CPLEX and Gurobi solvers) will most likely be...
---
QUBO model whose significance will be made clearer by numerical examples that give a sense of the diverse array of practical QUBO applications. Definition:  The QUBO model is expressed by the optimization problem:  QUBO:  minimize/maximize 𝑦=	𝑥%𝑄𝑥  where x is a vector of binary decision variables...
---
without needing to modify the coefficients by the approach shown in Section 1.      5. Other than the 0/1 restrictions on the decision variables, QUBO is an unconstrained model with all problem data being contained in the Q matrix.  These characteristics make the QUBO model particularly attractive...
---


Najbardziej podobne fragmenty pochodzą z 1  dokumentu który zawarłem co ma sens bo on jest wstępem do problemu, fragmenty te nie są żadnym podsumowaniem co ma sens bo wyszukujemy fragmenty dokumentów. Odpowiadając na pytanie "Po co nam baza wektorowa" jest potrzebna do przechowywania embeddingów, czyli wektorowych reprezentacji tekstów, obrazów czy innych danych, w przestrzeni wielowymiarowej (u nas tekst). Dzięki temu możemy wyszukiwać nie tylko po słowach kluczowych, ale wpisując też całe zdania co zrobiliśmy u góry bo brana pod uwagę jest też semantyka. Mechanizm ten opiera się na obliczaniu podobieństwa między wektorami ( są różne miary do podobieństw). Teraz możemy przejść do uzyskania odpowiedzi za pomocą lokalnego LLM.

## generowanie odpowiedzi z wykorzystaniem lokalnego modelu LLM (Hugging Face)
Wykorzystamy sobie najmniejszą wersję modelu flan (mam słaby internet by pobierać większe)

In [7]:
model_name = "google/flan-t5-small"
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

pipe = pipeline(
    "text2text-generation",
    model=model,
    tokenizer=tokenizer,
    max_length=1024
)

llm = HuggingFacePipeline(pipeline=pipe)

# Stworzenie łańcucha RAG
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=db.as_retriever()
)

result = qa_chain.invoke("How can we combine qubo and graph neural network in optimization")
print(f"AI Assistant: {result['result']}")

Device set to use cuda:0
C:\Users\jakub\AppData\Local\Temp\ipykernel_18828\1502518394.py:12: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=pipe)


AI Assistant: Combinatorial Optimization. The field of the model. • As previously emphasized, a variety of optimization problems can naturally be formulated and solved as an instance of the QUBO model. In addition, many other problems that don’t appear to be related to QUBO problems can be re-formulated as a QUBO model. We illustrate this reformulating important optimization problems as QUBO models through a series of explicit examples. Collectively these examples highlight the application breadth of the QUBO model. We disclose the unexpected advantages of modeling a wide range of problems in a form that differs from the linear h k  1  FIG. 1. Schematic illustration of the graph neural network approach for combinatorial optimization presented in this work. Following a recursive neighborhood aggregation scheme, thegraphneuralnetworkisiterativelytrainedagainst a custom loss function that encodes the specific optimization


Tutaj akurat dostaliśmy odpowiedź niezbyt poprawną i opcja wyszukania fragmentów byłaby lepsza gdyż też odpowiedź nie jest za prosta.

## implementacja mechanizmu pętli + pamięć konwersacji
żeby to zrobić musimy skorzystać z chaina ConversationalRetrievalChain by mieć pętle oraz ConversationBufferMemory by mieć pamięć

In [8]:
model_name = "google/flan-t5-small"
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

pipe = pipeline(
    "text2text-generation",
    model=model,
    tokenizer=tokenizer,
    max_length=1024
)

llm = HuggingFacePipeline(pipeline=pipe)

# pamięć
memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True
)

# Stworzenie łańcucha RAG
qa_chain = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=db.as_retriever(),
    memory=memory
)

while True:
    query = input("You: ")
    print(f"{query}")
    if query.lower() in ["exit", "quit"]:
        break
    result = qa_chain.invoke({"question": query})
    print(f"Asystent: {result['answer']}")

Device set to use cuda:0
C:\Users\jakub\AppData\Local\Temp\ipykernel_18828\2219600613.py:15: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(


Describe GNN
Asystent: GNNs are a family of neural networks capable of learning how to aggregate information in graphs for the purpose of representation learning. Typically, a GNN layer is comprised of three functions [35]: (i) a message WhileseveralspecificimplementsofGNNsexist[29, 41, 42], at their core typically GNNs iteratively update the features of the nodes of a graph by aggregating the information from their neighbors (often referred to as message passing [43]) thereby iteratively making local Intuitively, the goal of a graph neural network(GNN) is to embed information contained in a graph (e.g., the structure of the graph, spatial properties, features of the nodes, etc.) into a d-dimensional tensor for each nodeuV of the graph. To do so, information on a node  Rd0 , usually derived from the node’s label or given input features of dimensionality d0 [68]. Following a recursive neighborhood aggregation scheme, the GNN then iteratively updates each node’s representation, in genera

Token indices sequence length is longer than the specified maximum sequence length for this model (658 > 512). Running this sequence through the model will result in indexing errors


How to combine QUBO plus GNN
Asystent: [III. PRELIMINARIES]
remind me what i asked earlier
Asystent: GgUCO8Hkbu53nlBpHssHM03Qj+hI8pAzaqzU9AalsltxFyDrxMtJGXI0BqWv/jBmaYTSMEG17nluYvyMKsOZwFmxn2pMKJvQEfYslTRC7WeLQ2fk0ipDEsbKljRkof6eyGik9TQKbGdEzVivenPxP6+XmvDWz7hAq7Agxuowz00oAUMEJ7hFd6cR+fFeXc+lq0bTj5zBn/gfP4AeLGMrg==/latexit> Ky/DGlBBsGITTRAWVGe18AgJhJWurKJLcOa/vEja9ZpzXqvfnlUb10UdZTiEIzgBBy6gATfQhBZgeIRneIU348l4Md6Nj9loySh29uEPjM8fjLeWcw==/latexit> GgUCO8Hkbu53nlBpHssHM03Qj+hI8pAzaqzU9AalsltxFyDrxMtJGXI0BqWv/jBmaYTSMEG17nluYvyMKsOZwFmxn2pMKJvQEfYslTRC7WeLQ2fk0ipDEsbKljRkof6eyGik9TQKbGdEzVivenPxP6+XmvDWz7hAgxuowz00oAUMEJ7hFd6cR+fFeXc+lq0bTj5zBn/gfP4AeLGMrg==/latexit> GgUCO8Hkbu53nlBpHssHM03Qj+hI8pAzaqzU9AalsltxFyDrxMtJGXI0BqWv/jBmaYTSMEG17nluYvyMKsOZwFmxn2pMKJvQEfYslTRC7WeLQ2fk0ipDEsbKljRkof6eyGik9TQKbGdEzVivenPxP6+XmvDWz7hAgxuowz00oAUMEJ7hFd6cR+fFeXc+lq0bTj5zBn/gfP4AeLGMrg==/latexit> GgUCO8Hkbu53nlBpHssHM03Qj+hI8pAzaqzU9AalsltxFyDrxMtJGXI0BqWv/jBmaYTSMEG17nluYvyMKsOZwFmxn2pMKJvQEfYslTRC

model ma problemy ze zbyt długimi pytaniami oraz gdy wyjdziemy z chatu przedostatnia odpowiedź modelu to nonsens

# Wnioski
- modele typu transformer świetnie nadają się do zadań przetwarzania języka naturalnego
- model flan-t5-small ma problemy ze zbyt długimi promptami (pewnie jego większe wersje nie mają tego problemu i są bardziej dokładne)
- w przypadku bardziej skomplikowanych artykułów opcja znajdowania fragmentów jest o wiele bardziej przydatna niż podsumowania generowane przez llm które najczęściej są niepoprawne
- wynik "dobry" to taki który idealnie przywołuję dane fragmenty tekstu albo poprawnie podsumowuje tekst